# Cosmosdb connection testing

## Azure python documentation
https://learn.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos?preserve-view=true&view=azure-python


In [1]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
import os
import json
from dotenv import load_dotenv
from azure.cosmos import CosmosClient, PartitionKey

load_dotenv('../../shared/.env')

True

In [2]:
# database env
URL = os.getenv('COSMOSDB_ENDPOINT')  
MASTER_KEY = os.getenv('COSMOSDB_KEY')

In [3]:
# init client
client = cosmos_client.CosmosClient(URL, {'masterKey': MASTER_KEY}, user_agent="Andy", user_agent_overwrite=True)


In [4]:
# Create a database (if it doesn't exist)
database_name = 'MojovaDB'
database = client.get_database_client(database_name)

# Create a container (if it doesn't exist)
container_name = 'Analytiikka'
try:
    container = database.create_container_if_not_exists(
        id=container_name,
        partition_key=PartitionKey(path='/TestiOsio'),
        offer_throughput=400
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

In [5]:
# read local datafile
datafile = '../../data/ingestion/117_HOpp-Q3_2023.xlsx.json'

with open(datafile, "r") as f:
    content = f.read()
    content = json.loads(content)
    print(content)


[
    {
        "organisaatiokoodi_kolme_numeroa":117,
        "yksikkokoodiks_luokitteluohje_yksikkokoodit-valilehdelta":"AIKKIRMUU",
        "kvartaali_ja_vuosiesim_1_2020":"3_2023",
        "1_hoitajat_ottivat_mielipiteeni_huomioon_kun_hoitoani_suunniteltiin_tai_toteutettiin":5,
        "2_hoitajat_ja_laakarit_toimivat_hyvin_yhdessa_hoitooni_liittyvissa_asioissa":5,
        "3_hoitoni_oli_hyvin_suunniteltu_ja_toteutettu_hoitajien_seka_laakareiden_toimesta":5,
        "4_hoitajat_pyysivat_minulta_anteeksi_jos_hoidossani_tapahtui_virhe":5,
        "5_hoitajat_puuttuivat_epakohtaan_josta_mainitsin_heille":5,
        "6_hoitajat_kertoivat_minulle_uuden_laakkeen_antamisen_yhteydessa_miksi_laaketta_annetaan":5,
        "7_hoitajat_kertoivat_minulle_saamieni_laakkeiden_mahdollisista_sivuvaikutuksista":5,
        "8_hoitajat_puhuivat_arkaluontoisista_asioista_siten_etteivat_ulkopuoliset_kuulleet_niita":5,
        "9_hoitajat_huolehtivat_etta_liikkuminen_oli_turvallista_hoidon_aikana":5,
   

In [6]:
# add data to container
import uuid
for item in content:
    item.setdefault("id", str(uuid.uuid4()))
    container.upsert_item(item)

AttributeError: 'str' object has no attribute 'setdefault'